# Identifica produtos contidos em cada venda

Leitura do DataFrame de vendas e de custos

In [11]:
import pandas as pd
from akira.utils import replace_caractere_decimal

In [2]:
vendas_df = pd.read_csv('../data/gold/2025_11_final.csv')
custos_df = pd.read_excel('../data/bronze/custos_akira.xlsx')

# Altera nome das colunas de custos_df
custos_df.columns = ['descricao', 'custo']

Segmenta DataFrame de custos

In [3]:
lista_produtos = ['STR/BZ', 'AKAI/FEM', 'FX']

In [4]:
custos_produtos_df = custos_df[custos_df['descricao'].isin(lista_produtos)]
outros_custos_df = custos_df[~custos_df['descricao'].isin(lista_produtos)]

In [5]:
# Cria dicionário com valores por produto
custos_produtos_dict = dict(zip(custos_produtos_df['descricao'], custos_produtos_df['custo']))

# Identifica produtos em cada venda

Seleciona colunas e filtra linhas de interesse no df de vendas

In [6]:
# Define lista de colunas para select
colunas_select = ['id_venda', 'unidades_vendidas', 'titulo_anuncio', 'count_produto']

# Seleciona colunas e filtra
vendas_df = vendas_df[colunas_select]
vendas_df = vendas_df[vendas_df['count_produto']==True]

# Remove a coluna count_produto, que não será mais necessária
colunas_select.remove('count_produto')
vendas_df = vendas_df[colunas_select]

Cria DataFrame de produtos por venda

In [7]:
# Esse dataframe dividirá uma linha de venda em mais linhas representando os produtos contidos na venda
produtos_por_venda_df = pd.DataFrame(columns=['id_venda', 'unidades_vendidas', 'categoria_produto', 'custo_producao'])

Itera pelo DataFrame de vendas para identificar os produtos de cada venda

In [8]:
for row in vendas_df.itertuples():

    # Define lista auxiliar para armazenar produtos da venda e título do anúncio
    lista_produtos = []
    titulo_anuncio = row.titulo_anuncio.lower()

    # De acordo com os produtos encontrados no título do anúncio, adiciona produto na lista de produtos
    if ('akai' in titulo_anuncio) or ('feminin' in titulo_anuncio):
        lista_produtos.append('AKAI/FEM')
    else:
        lista_produtos.append('STR/BZ')
    if 'faixa' in titulo_anuncio:
        lista_produtos.append('FX')

    # Para cada produto, cria uma linha no df de produtos por venda
    for produto in lista_produtos:
        produtos_por_venda_df.loc[len(produtos_por_venda_df)] = {
            'id_venda': row.id_venda,
            'unidades_vendidas': row.unidades_vendidas,
            'categoria_produto': produto,
            'custo_producao': row.unidades_vendidas * custos_produtos_dict[produto]
        }

Salva DataFrames

In [12]:
# Aplica a correção de caractere decimal em cada dataframe
outros_custos_df = replace_caractere_decimal(outros_custos_df)
produtos_por_venda_df = replace_caractere_decimal(produtos_por_venda_df)

# Salva
outros_custos_df.to_csv('../data/gold/outros_custos.csv')
produtos_por_venda_df.to_csv('../data/gold/produtos_por_venda.csv')

C:\Users\yzalm\Projetos\akira\src\akira\utils.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].fillna(0).astype(str).str.replace('.', ',', regex=False)
